In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import numpy as np

In [1]:
#Extracting Features using a Pre-Trained ResNet Model
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final fully connected layer

    def forward(self, x):
        return self.resnet(x)

#Setting up a shallow Fully Connected Classifier
class Classifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = torch.flatten(x, 1)  # Flatten the features
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#Defining the Data Trasnformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset_root = 'blur_dataset_scaled/'
dataset = datasets.ImageFolder(dataset_root, transform=transform)
num_samples = len(dataset)

#Data Set Split is 275 for Train, 25 for Validation, 50 for test
indices = list(range(num_samples))
np.random.shuffle(indices)
split1 = int(0.275 * num_samples)
split2 = int(0.025 * num_samples)

train_indices = indices[:split1]
val_indices = indices[split1:split1+split2]
test_indices = indices[split1+split2:]

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

#Data Loader & Feature Extraction usage
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

feature_extractor = FeatureExtractor()
classifier = Classifier(512, 3)  # ResNet18 has 512 output features
model = nn.Sequential(feature_extractor, classifier)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#model Training Loop (uasing the parameters set)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        features = feature_extractor(images)
        outputs = classifier(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    #Epoch-wise Evaluation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for images, labels in val_loader:
            features = feature_extractor(images)
            outputs = classifier(features)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {correct/total}")


#Evaluating the model - Post Training with 10 Epochs
model.eval()
with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0
    for images, labels in test_loader:
        features = feature_extractor(images)
        outputs = classifier(features)
        test_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Loss: {test_loss/len(test_loader)}, Test Accuracy: {correct/total}")

# Save the model
torch.save(model.state_dict(), 'model.pth')

C:\Users\Maaz Ali\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Maaz Ali\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 4.117355823516846, Accuracy: 0.34615384615384615
Epoch 2/10, Loss: 2.6340746879577637, Accuracy: 0.4230769230769231
Epoch 3/10, Loss: 1.694183588027954, Accuracy: 0.5384615384615384
Epoch 4/10, Loss: 14.550994873046875, Accuracy: 0.2692307692307692
Epoch 5/10, Loss: 3.1417717933654785, Accuracy: 0.5
Epoch 6/10, Loss: 1.7035554647445679, Accuracy: 0.6153846153846154
Epoch 7/10, Loss: 0.898887574672699, Accuracy: 0.6538461538461539
Epoch 8/10, Loss: 0.7679473161697388, Accuracy: 0.6538461538461539
Epoch 9/10, Loss: 1.342573881149292, Accuracy: 0.6538461538461539
Epoch 10/10, Loss: 3.1368520259857178, Accuracy: 0.38461538461538464
Test Loss: 3.028779112774393, Test Accuracy: 0.49320652173913043


In [2]:
#Defining an Inference Function
def classify_image(image_path, model):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    image = transform(Image.open(image_path)).unsqueeze(0)
    features = feature_extractor(image)
    outputs = classifier(features)
    _, predicted = torch.max(outputs.data, 1)
    return predicted.item()

In [6]:
from PIL import Image
category = {0: "blur", 1:"blur", 2:"sharp"}
image_path = 'img2.jpeg'
model.load_state_dict(torch.load('model.pth'))
model.eval()
class_index = classify_image(image_path, model)
print(category[class_index])

sharp
